# 回転の推定I:等方性誤差

In [ ]:
import sys

import numpy as np
from scipy.stats import special_ortho_group
from scipy.spatial.transform import Rotation

sys.path.append('../libs')
import util

In [ ]:
A = util.load_point_cloud()
util.plot_3d(A)

In [ ]:
R = special_ortho_group.rvs(3)
print(R)

In [ ]:
noise = np.random.normal(0, 3e-3, A.shape)
A_prime = R @ A + noise
util.plot_3d(A_prime)

## 4.3 特異値分解による解法

In [ ]:
estimated_R = util.estimate_R_using_SVD(A, A_prime)
print(util.eval_R_error(estimated_R, R))

In [ ]:
util.plot_3d_multi(estimated_R @ A, A_prime)

## 4.4 四元数表示による解法

In [ ]:
N = A @ A_prime.T
N_tilde = np.array([
    [
        N[0, 0] + N[1, 1] + N[2, 2],
        -N[2, 1] + N[1, 2],
        N[2, 0] - N[0, 2],
        -N[1, 0] + N[0, 1],
    ], [
        -N[2, 1] + N[1, 2],
        N[0, 0] - N[1, 1] - N[2, 2],
        N[1, 0] + N[0, 1],
        N[2, 0] + N[0, 2],
    ], [
        N[2, 0] - N[0, 2],
        N[1, 0] + N[0, 1],
        -N[0, 0] + N[1, 1] - N[2, 2],
        N[2, 1] + N[1, 2],
    ], [
        -N[1, 0] + N[0, 1],
        N[2, 0] + N[0, 2],
        N[2, 1] + N[1, 2],
        -N[0, 0] - N[1, 1] + N[2, 2],
    ],
])

In [ ]:
w, v = np.linalg.eig(N_tilde)
max_vec = v[:, np.argmax(w)]
# 書籍では(w, x, y, z)だが，scipyでは(x, y, z, w)
r = Rotation.from_quat(max_vec[[1, 2, 3, 0]])
print(util.eval_R_error(r.as_dcm(), R))

In [ ]:
util.plot_3d_multi(r.as_dcm() @ A, A_prime)

## 4.5 回転行列の最適補正

In [ ]:
R_hat = R + np.random.normal(0, 1e-2, R.shape)
R_hat @ R_hat.T

In [ ]:
U, S, VT = np.linalg.svd(R_hat)
V = VT.T
corrected_R = U @ np.diag([1, 1, np.linalg.det(U @ V)]) @ VT
corrected_R @ corrected_R.T